In [1]:
import rasterio
from glob import glob
import numpy as np
import pandas as pd
import pickle
import os


In [2]:
##### START OPTIONS #####
root_path = "/data/"
stats_folder = root_path + "band_stats/"
chip_dir_filt = root_path + 'chips_filtered/'
chip_dir_binary_filt = root_path + 'chips_binary_filtered/'
chip_dir_multi_filt = root_path + 'chips_multi_filtered/'

#####  END OPTIONS  #####

In [3]:
dirs_to_make = [stats_folder]
for folder in dirs_to_make:
    try:
        os.makedirs(folder)
    except FileExistsError:
        # directory already exists
        print('pass')
        pass

read in values for all HLS bands, one band at a time. save values on a per band basis

In [13]:
chips = glob(chip_dir_filt + "*_merged.tif")
print(len(chips))
means = []
stds = []

for band in range(1, 19):
    print(band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
     #   print(k)
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            vals_all = np.concatenate([vals_all, vals])
    mean = vals_all.mean()
    std = vals_all.std()
    print(mean)
    print(std)
    means.append(mean)
    stds.append(std)
    
    with open(stats_folder + 'band_values_' + str(band) + '.list', 'wb') as file:
        pickle.dump(vals_all, file)
        



        
    #out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)


82
1
475.98330535053196
146.46174845556203
2
777.0021543678447
175.13277578040547
3
995.5648259589659
340.6730466061487
4
2402.4183846032697
510.87817441986573
5
2834.097757600563
606.7183205001566
6
1994.2226003492099
614.3855352701568
7
404.06936461703583
154.57348214419721
8
716.4779340137351
179.76939525521266
9
838.4898632423625
353.10781836706155
10
2633.105708880351
472.1792105845257
11
2617.000048123289
630.3450079285017
12
1667.5238499506129
665.4820760040467
13
432.6193649573015
156.86260998036263
14
757.6270386775137
201.1481813904242
15
1043.4415907226075
364.76270076992
16
2654.7527816719294
501.581632457672
17
2949.277601136682
647.7870283209181
18
1954.2886707083749
703.3462607491523


In [14]:
print(means)
print(stds)

[475.98330535053196, 777.0021543678447, 995.5648259589659, 2402.4183846032697, 2834.097757600563, 1994.2226003492099, 404.06936461703583, 716.4779340137351, 838.4898632423625, 2633.105708880351, 2617.000048123289, 1667.5238499506129, 432.6193649573015, 757.6270386775137, 1043.4415907226075, 2654.7527816719294, 2949.277601136682, 1954.2886707083749]
[146.46174845556203, 175.13277578040547, 340.6730466061487, 510.87817441986573, 606.7183205001566, 614.3855352701568, 154.57348214419721, 179.76939525521266, 353.10781836706155, 472.1792105845257, 630.3450079285017, 665.4820760040467, 156.86260998036263, 201.1481813904242, 364.76270076992, 501.581632457672, 647.7870283209181, 703.3462607491523]


save band means and sds to csv

In [15]:
band_names = list(range(1, 19))
summary_df = pd.DataFrame({"band": band_names  ,  "mean": means, "sd": stds})
summary_df
summary_df.to_csv(stats_folder + "means_sds.csv")

consolidate means, sds for dates

In [16]:
means = []
stds = []

for item in [[1,7,13], [2,8,14], [3,9,15], [4,10,16], [5, 11, 17], [6, 12, 18]]:
    print(item)
    band_vals = []
    for band in item:
        print(band)
        with open(stats_folder + 'band_values_' + str(band) + '.list', 'rb') as f:
            x = pickle.load(f)
            print(x.mean())
            print(x.std())
            band_vals.append(x)
            del x

    band_vals = np.concatenate((band_vals[0], band_vals[1], band_vals[2]))
    mean = band_vals.mean()
    std = band_vals.std()
    means.append(mean)
    stds.append(std)
    del band_vals
        
            

[1, 7, 13]
1
475.98330535053196
146.46174845556203
7
404.06936461703583
154.57348214419721
13
432.6193649573015
156.86260998036263
[2, 8, 14]
2
777.0021543678447
175.13277578040547
8
716.4779340137351
179.76939525521266
14
757.6270386775137
201.1481813904242
[3, 9, 15]
3
995.5648259589659
340.6730466061487
9
838.4898632423625
353.10781836706155
15
1043.4415907226075
364.76270076992
[4, 10, 16]
4
2402.4183846032697
510.87817441986573
10
2633.105708880351
472.1792105845257
16
2654.7527816719294
501.581632457672
[5, 11, 17]
5
2834.097757600563
606.7183205001566
11
2617.000048123289
630.3450079285017
17
2949.277601136682
647.7870283209181
[6, 12, 18]
6
1994.2226003492099
614.3855352701568
12
1667.5238499506129
665.4820760040467
18
1954.2886707083749
703.3462607491523


In [17]:
print(means)
print(stds)

[437.55734497495644, 750.3690423530312, 959.165426641312, 2563.42562505185, 2800.125135620178, 1872.0117070027325]
[155.53377603014596, 187.4030628443723, 363.6780002756211, 508.1512718083158, 643.4297107937471, 677.8771341063208]


In [19]:
band_names = ['B02', 'B03', 'B04', 'B8A', 'B11', 'B12']
summary_df = pd.DataFrame({"band": band_names  ,  "mean": means, "sd": stds})
summary_df
summary_df.to_csv(stats_folder + "means_sds_consolidated_dates.csv")

calculate counts for all CDL classes

In [20]:
chips = glob(chip_dir_filt +  "*.mask.tif")
print(len(chips))
means = []
stds = []

for band in range(1, 2):
    print(band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
     #   print(k)
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            if(len(vals) != 50176):
                print(k)
                print(file)
                print(len(vals))
            vals_all = np.concatenate([vals_all, vals])
    mean = vals_all.mean()
    std = vals_all.std()
    print(mean)
    print(std)
    means.append(mean)
    stds.append(std)
    
    with open(stats_folder + 'CDL_values.list', 'wb') as file:
        pickle.dump(vals_all, file)
        


82
1
126.13948729739609
68.23033437177017


export CDL counts 

In [28]:
len(vals_all)
unique, counts = np.unique(vals_all, return_counts=True)
total_count = counts.sum()
pcts = (100.0*counts)/total_count
freq_table =  pd.DataFrame({"CDL_val": unique  ,  "count": counts, "pct": pcts})

In [29]:
freq_table.to_csv(stats_folder + "cdl_freq.csv")

In [30]:
counts.sum()

212645888